# Data prepping: cleaning & features engineering

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mode
import string
from functions.utilities import *

In [2]:
#to be able to have multiple output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#read the data
trainpath = './data/train.csv'
testpath = './data/test.csv'
traindf = pd.read_csv(trainpath, delimiter=",")
testdf = pd.read_csv(testpath, delimiter=",")
fulldf = traindf.merge(testdf, how="outer")

dfs = [fulldf, testdf, traindf]

#show the data set
fulldf.shape #returns (#lines, #columns)
fulldf.head()

#data type dictionary
data_type_dict={'PassengerId':'numeric',
                'Survived':'nominal',
                'Pclass':'ordinal',
                'Name':'nominal',
                'Sex':'nominal', 
                'Age':'numeric', 
                'SibSp':'numeric',
                'Parch':'numeric',
                'Ticket':'nominal',
                'Fare':'numeric', 
                'Cabin': 'nominal',
                'Embarked':'nominal'}

(1309, 12)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# check which columns have NA values
fulldf.isna().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

We can see from the last table that some columns requires cleaning: Cabin, Embarked and Age.

In [5]:
#get an understanding of the possible values in relevant columns
fulldf.Pclass.unique()
fulldf.Sex.unique()
fulldf.SibSp.unique()
fulldf.Parch.unique()

array([3, 1, 2], dtype=int64)

array(['male', 'female'], dtype=object)

array([1, 0, 3, 4, 2, 5, 8], dtype=int64)

array([0, 1, 2, 5, 3, 4, 6, 9], dtype=int64)

## Data cleaning

### Embarked

In [6]:
fulldf.Embarked.unique()
fulldf[fulldf.Embarked.isna()]

array(['S', 'C', 'Q', nan], dtype=object)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1.0,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1.0,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [7]:
fulldf.Cabin = fulldf.Cabin.fillna('Unknown')
fulldf.loc[(fulldf["Name"].str.contains('Icard|Stone')) | (fulldf["Cabin"].str.contains('B28'))]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1.0,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
319,320,1.0,1,"Spedden, Mrs. Frederic Oakley (Margaretta Corn...",female,40.0,1,1,16966,134.5,E34,C
829,830,1.0,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


### Cabin

In [8]:
fulldf.Cabin.unique()
# fulldf[fulldf.Cabin=="Unknown"] #we changed NA values to Unknown just before

array(['Unknown', 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62

In [9]:
def peoplePerCat(df,category):
    return df.loc[(df['Pclass'] == category)].shape[0]

def unknownCabinsPerCat(df,category):
    return df.loc[(df['Cabin'] == "Unknown") & (df['Pclass'] == category)].shape[0]
 
categories = [1, 2, 3]
 
nb_unknown = fulldf.loc[(fulldf['Cabin'] == "Unknown")].shape[0] #1014
nb_unknownPerCat = list(map(lambda x: unknownCabinsPerCat(fulldf, x),categories))
nb_perCat = list(map(lambda x: peoplePerCat(fulldf, x), categories))
    
np.divide(nb_unknownPerCat,nb_perCat)

array([0.20743034, 0.91696751, 0.977433  ])

### Age

In [10]:
fulldf.Age.describe()
fulldf[fulldf.Age.isna()]

count    1046.000000
mean       29.881138
std        14.413493
min         0.170000
25%        21.000000
50%        28.000000
75%        39.000000
max        80.000000
Name: Age, dtype: float64

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0.0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,Unknown,Q
17,18,1.0,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,Unknown,S
19,20,1.0,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,Unknown,C
26,27,0.0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,Unknown,C
28,29,1.0,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,Unknown,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
1299,1300,NaN,3,"Riordan, Miss. Johanna Hannah""""",female,NaN,0,0,334915,7.7208,Unknown,Q
1301,1302,NaN,3,"Naughton, Miss. Hannah",female,NaN,0,0,365237,7.7500,Unknown,Q
1304,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,Unknown,S
1307,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,Unknown,S


## Fare and category

In [11]:
fulldf.Fare.describe()
fulldf[fulldf.Fare.isna()]

count    1308.000000
mean       33.295479
std        51.758668
min         0.000000
25%         7.895800
50%        14.454200
75%        31.275000
max       512.329200
Name: Fare, dtype: float64

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1043,1044,NaN,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,Unknown,S


## Finding the mean and variance

In [12]:
# Description: compute the mean and std of the fram of each category
# Input: dataframe df, list of categories
# Output: list of tuples giving the mean and the variance for each cat
def statFaresCat(df, categories):
    statInfos = []
    for category in categories:
        dfCat = df.loc[df['Pclass'] == category]
        df_stats = [dfCat.loc[:,"Fare"].describe()]
        statInfos.append(df_stats)
    return statInfos 
                         
statFaresCat(fulldf, categories)

[[count    323.000000
  mean      87.508992
  std       80.447178
  min        0.000000
  25%       30.695800
  50%       60.000000
  75%      107.662500
  max      512.329200
  Name: Fare, dtype: float64],
 [count    277.000000
  mean      21.179196
  std       13.607122
  min        0.000000
  25%       13.000000
  50%       15.045800
  75%       26.000000
  max       73.500000
  Name: Fare, dtype: float64],
 [count    708.000000
  mean      13.302889
  std       11.494358
  min        0.000000
  25%        7.750000
  50%        8.050000
  75%       15.245800
  max       69.550000
  Name: Fare, dtype: float64]]

As we can see, some people have a fare of zero. Let's see how many.

In [13]:
for category in categories :
    print("# person whose fare is zero in category " + str(category) + ": " + str(len(fulldf.loc[(fulldf["Fare"]== 0) & (fulldf["Pclass"] == category)])))
    
fulldf.loc[(fulldf["Fare"]== 0) & fulldf["Cabin"].notna()]

# person whose fare is zero in category 1: 7
# person whose fare is zero in category 2: 6
# person whose fare is zero in category 3: 4


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
179,180,0.0,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0,Unknown,S
263,264,0.0,1,"Harrison, Mr. William",male,40.0,0,0,112059,0.0,B94,S
271,272,1.0,3,"Tornquist, Mr. William Henry",male,25.0,0,0,LINE,0.0,Unknown,S
277,278,0.0,2,"Parkes, Mr. Francis ""Frank""",male,NaN,0,0,239853,0.0,Unknown,S
302,303,0.0,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0,Unknown,S
413,414,0.0,2,"Cunningham, Mr. Alfred Fleming",male,NaN,0,0,239853,0.0,Unknown,S
466,467,0.0,2,"Campbell, Mr. William",male,NaN,0,0,239853,0.0,Unknown,S
481,482,0.0,2,"Frost, Mr. Anthony Wood ""Archie""",male,NaN,0,0,239854,0.0,Unknown,S
597,598,0.0,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0,Unknown,S
633,634,0.0,1,"Parr, Mr. William Henry Marsh",male,NaN,0,0,112052,0.0,Unknown,S


### Inducing the fare from the category

In [14]:
def addFareGivenCat(df, category, categories, fulldf):
    statInfos = statFaresCat(fulldf,categories)

### Verifying the category from the fare

### Verifying members from a same family have the same category

### Getting the fare per person

## Features engineering
### Getting the title from the name

In [15]:
title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev','Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess', 'Don', 'Jonkheer']

female_title_list = ['Mrs', 'Miss', 'Ms', 'Mlle', 'Mme', 'Countess']
male_title_list = ['Mr', 'Master', 'Major', 'Rev', 'Col', 'Don', 'Jonkheer', 'Capt']
neutral_title_list = ['Dr']

#TODO remove captain from dataset as it was a choice and not related to its status

def addColumnTitle(df, title_list):
    df['Title']=df['Name'].map(lambda x: substrings_in_string(x, title_list))
    data_type_dict['Title'] = 'nominal'
    
updateAllDataSets(addColumnTitle, dfs, title_list)

### Getting the Deck from the cabin number

In [16]:
#Turning cabin number into Deck
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'Unknown'] 

def addColumnDeck(df, cabin_list):
    df.Cabin = df.Cabin.fillna('Unknown')
    df['Deck']=df['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
    
updateAllDataSets(addColumnDeck, dfs, cabin_list)
#fulldf.head()

## Output

In [17]:
testdf.to_csv("testdf-2.csv", index=False)
traindf.to_csv("traindf-2.csv",index=False)